In [1]:
import numpy as np
import pandas as pd
from scipy.stats import poisson, norm
from sklearn.linear_model import LinearRegression

In [2]:
## loading datasets
staking_flow =  pd.read_csv('model/data/staking_flow.csv')
eth_burn = pd.read_csv('model/data/eth_burn.csv')

cumu_staked = pd.read_csv('model/data/cumu_staked_validator.csv')

In [4]:
print(staking_flow)

       amount_full_withdrawn  amount_partial_withdrawn  amount_staked  \
0                        0.0                 -5.947760              0   
1                        0.0                 -1.515562              0   
2                        0.0                  0.000000             64   
3                     -160.0                -26.890801              0   
4                        0.0                 -8.772212              0   
...                      ...                       ...            ...   
26463                    0.0                  0.000000            224   
26464                    0.0                  0.000000             96   
26465                    0.0                  0.000000           8864   
26466                    0.0                  0.000000           8096   
26467                    0.0                  0.000000           2112   

              day          entity entity_category sub_entity  \
0      2024-08-21           BTC-e            CEXs          

In [3]:
recent_staked = cumu_staked[cumu_staked['day']==cumu_staked['day'].max()]
print(recent_staked)
curr_stake = recent_staked[recent_staked['entity_category']=='Liquid Restaking']['curr_stake']

   amount_full_withdrawn  amount_partial_withdrawn  amount_staked  \
0                  960.0                171.572760              0   
1                 3680.0                123.906669           1504   
2                 2272.0                175.889942           1216   
3                  736.0                270.989009           1696   
4                   32.0                  0.383541              0   
5                  672.0                247.529829          15185   

   cumulative_amount_full_withdrawn  cumulative_amount_partial_withdrawn  \
0                      1.075039e+06                         39889.195288   
1                      1.776657e+06                        368449.555773   
2                      6.394754e+06                        803357.335542   
3                      3.853881e+06                        329806.532933   
4                      1.202491e+05                         44379.896161   
5                      3.104268e+06                        7

In [45]:
staking_flow['amount_full_withdrawn'] = -staking_flow['amount_full_withdrawn']/32
staking_flow['amount_partial_withdrawn'] = -staking_flow['amount_partial_withdrawn']

staking_flow['amount_staked'] = staking_flow['amount_staked']/32

In [50]:
def simulate_data(df, n_samples, past_days, simulated_type, simulated_name):
    # Filter and potentially aggregate the data
    if simulated_type == 'entity':
        filtered_data = df[df[simulated_type] == simulated_name]
    elif simulated_type == 'entity_category':
        filtered_data = df[df[simulated_type] == simulated_name]
        filtered_data = filtered_data.groupby('day').agg({
            'amount_full_withdrawn': 'sum',
            'amount_partial_withdrawn': 'sum',
            'amount_staked': 'sum'
        }).reset_index()
        filtered_data['day'] = pd.to_datetime(filtered_data['day'])
    else:
        raise ValueError("simulated_type must be 'entity' or 'entity_category'")

    # Filter data to include only the last 'past_days'
    max_date = filtered_data['day'].max()
    start_date = max_date - pd.Timedelta(days=past_days)
    filtered_data = filtered_data[filtered_data['day'] > start_date]

    # Perform simulations
    simulated_withdrawn = simulate_zero_inflated_data(filtered_data['amount_full_withdrawn'], n_samples, (filtered_data['amount_full_withdrawn'] == 0).mean())
    simulated_staked = simulate_zero_inflated_data(filtered_data['amount_staked'], n_samples, (filtered_data['amount_staked'] == 0).mean())
    simulated_partial_withdrawn = simulate_based_on_day(filtered_data, n_samples)

    # Make withdrawals negative
    simulated_withdrawn = -np.abs(simulated_withdrawn)
    simulated_partial_withdrawn = -np.abs(simulated_partial_withdrawn)

    # Compile all simulated data into a DataFrame
    simulated_data = pd.DataFrame({
        'day': [max_date + pd.Timedelta(days=i) for i in range(1, n_samples + 1)],
        'amount_full_withdrawn': simulated_withdrawn,
        'amount_partial_withdrawn': simulated_partial_withdrawn,
        'amount_staked': simulated_staked
    })

    return simulated_data

def simulate_zero_inflated_data(data, n_samples, proportion_zeros):
    n_zeros = int(n_samples * proportion_zeros)
    zeros = np.zeros(n_zeros)
    non_zeros = poisson.rvs(mu=data[data > 0].mean(), size=n_samples - n_zeros)
    full_sample = np.concatenate([zeros, non_zeros])
    np.random.shuffle(full_sample)
    return full_sample

def simulate_based_on_day(data, n_samples):
    model = LinearRegression()
    days = np.array(data['day'].map(pd.Timestamp.toordinal)).reshape(-1, 1)
    values = data['amount_partial_withdrawn']
    model.fit(days, values)
    max_day = days.max()
    future_days = np.array([max_day + i for i in range(1, n_samples + 1)]).reshape(-1, 1)
    return model.predict(future_days)

In [51]:
simulated_data = simulate_data(staking_flow, 100, 300, 'entity_category', 'CEXs')  # 1000 samples, based on the last 30 days
print(simulated_data)

          day  amount_full_withdrawn  amount_partial_withdrawn  amount_staked
0  2024-08-22                 -395.0               -589.180067          371.0
1  2024-08-23                 -407.0               -588.676071          389.0
2  2024-08-24                 -373.0               -588.172076          375.0
3  2024-08-25                 -410.0               -587.668080          374.0
4  2024-08-26                 -432.0               -587.164084          440.0
..        ...                    ...                       ...            ...
95 2024-11-25                 -497.0               -541.300489          377.0
96 2024-11-26                 -391.0               -540.796493          383.0
97 2024-11-27                 -412.0               -540.292498          416.0
98 2024-11-28                 -409.0               -539.788502          415.0
99 2024-11-29                 -435.0               -539.284507          385.0

[100 rows x 4 columns]


In [53]:
staking_flow['entity_category'].unique()

array(['CEXs', 'Staking Pools', 'Solo Stakers', 'Liquid Staking', '0',
       'Liquid Restaking'], dtype=object)

In [57]:
simulated_data.iloc[0]['amount_full_withdrawn']

-395.0

In [58]:
staking_flow['amount_partial_withdrawn']

0         5.947760
1         1.515562
2        -0.000000
3        26.890801
4         8.772212
           ...    
26463    -0.000000
26464    -0.000000
26465    -0.000000
26466    -0.000000
26467    -0.000000
Name: amount_partial_withdrawn, Length: 26468, dtype: float64